# JalanJalan.ai: Your trusted AI Agent Personal Vacation in Indonesia
“Smart travel, local vibes, powered by AI.”

### by **Danang Agung Restu Aji**

#### Code Reviewer | NLP Research Engineer

[![LinkedIn](https://img.shields.io/badge/LinkedIn-Profdara-blue?logo=linkedin)](https://linkedin.com/in/Profdara)

Dari pulau jawa ke dunia wkwkwkwkw

JalanJalan.ai is a simple AI-powered travel assistant designed to help explorers discover Indonesia’s hidden gems. From weather-aware destination recommendations to budget planning, JalanJalan.ai coordinates multiple intelligent agents to deliver a seamless vacation plan.

- Location-aware suggestions
- Weather-based destination filtering
- Budget calculation with entrance fees & total cost
- Personalized itineraries for every travel


### Mission
To make traveling in Indonesia effortless, personalized, and unforgettable by blending AI intelligence with local culture.

### Problem & Solution Pitch

Planning a vacation in Indonesia can be overwhelming due to the abundance of destinations, unpredictable weather, and the challenge of budgeting for entrance fees and travel costs. Travelers often spend hours researching, comparing options, and organizing itineraries, which detracts from the excitement of the trip.

**Solution:**
build an AI-powered Vacation Planning Agent that automates destination discovery, weather-aware recommendations, and budget estimation. This agent will streamline the entire planning process, delivering personalized, actionable travel plans in minutes - making vacations effortless, enjoyable, and tailored to each explorer’s needs.

## Agent Initialization

This section sets up the environment and initializes the required agents for the JalanJalan.ai travel assistant. It includes loading API keys, importing necessary modules, and defining the core agent components used throughout the notebook.

In [62]:
# Install required packages for this project if it's local
import os
from dotenv import load_dotenv

load_dotenv()  # to read file .env , put your api key in .env file please
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

#import os
#from kaggle_secrets import UserSecretsClient

#user_secrets = UserSecretsClient()
#api_key = user_secrets.get_secret("GOOGLE_API_KEY")

#os.environ["GOOGLE_API_KEY"] = api_key


if not GOOGLE_API_KEY:
    print("API key not found")
else:
    print("✅ API key loaded:", GOOGLE_API_KEY[:5] + "*****")  # show partial to check

✅ API key loaded: AIzaS*****


In [63]:
# please pip install if not already installed

# %pip install google-adk


from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [64]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504], # Retry on these HTTP errors
)

In [65]:
root_agent = Agent(
    name="vacation_assistant",
    model=Gemini(
        model="gemini-2.5-flash-lite",
    ),
    description="A simple agent that get the data from user to be passed to other agents to create a complete travel plan.",
    instruction="You are a helpful assistant. Use Google Search for current info or if unsure, you task is to send the data to other agents to create a complete travel plan for a vacation.",
    tools=[google_search],
)
print("✅ root Agent defined.")

✅ root Agent defined.


In [66]:
runner = InMemoryRunner(agent=root_agent)

print("✅ Runner created.")

✅ Runner created.


In [67]:
response = await runner.run_debug(
    "What is Agent Development Kit from Google? What languages is the SDK available in?"
)


 ### Created new session: debug_session_id

User > What is Agent Development Kit from Google? What languages is the SDK available in?
vacation_assistant > The Agent Development Kit (ADK) from Google is a flexible and modular framework designed to simplify the development, deployment, and orchestration of AI agents and multi-agent systems. It applies software development principles to AI agent creation, allowing developers to build sophisticated agentic architectures that can range from simple tasks to complex workflows. The ADK is optimized for Gemini and the Google ecosystem but is designed to be model-agnostic and deployment-agnostic, offering compatibility with other frameworks.

The ADK is available in the following programming languages:
*   **Python**
*   **Java**
*   **Go**

The kit provides a rich tool ecosystem, code-first development for flexibility and testability, and support for modular multi-agent systems. It also includes a built-in development UI for testing and debugg

In [68]:
# testing weather in Yogyakarta
response = await runner.run_debug("What's the weather in Yogyakarta?")


 ### Continue session: debug_session_id

User > What's the weather in Yogyakarta?
vacation_assistant > The weather in Yogyakarta is currently cloudy with a temperature of 75°F (24°C). There is a 0% chance of rain, and the humidity is around 92%.

For the rest of the day, it is expected to remain cloudy with a 35% chance of rain. The temperature will range between 75°F (24°C) and 86°F (30°C).

Looking ahead, the forecast for the next few days includes a mix of light rain, thunderstorms, and mostly cloudy conditions. Temperatures are expected to stay warm, generally in the low to mid-80s Fahrenheit.


# Agent Members

## 1. weather and location agent


This agent specializes in retrieving up-to-date weather and location information using the `google_search` tool.  
It is designed to answer queries about current weather conditions and local details for any specified area.

- **Purpose:** Provide accurate weather and location context for travel planning.
- **How it works:** Uses Google Search to fetch and present findings based on user queries.
- **Output:** Clear, concise weather and location summaries.

In [69]:
# weather and location agent: Its job is to use the google_search tool and present findings.
location_weather_agent = Agent(
    name="location_weather_agent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a specialized location and weather agent. Your only job is to use the
    google_search tool to find the nearby weather and location information based on the user's query.""",
    tools=[google_search],
    output_key="location_weather_findings",  # The result of this agent will be stored in the session state with this key.
)

print("✅ location_weather_agent created.")

✅ location_weather_agent created.


## 2. destination agent


This agent specializes in discovering and summarizing nearby tourist attractions based on the user's current location and weather conditions.

- **Purpose:** Suggest relevant destinations, filtering out options unsuitable for current weather.
- **How it works:** Uses Google Search to find attractions, then summarizes each with:
    - Name (in English)
    - Type (landmark, nature, cultural site, etc.)
    - Distance or estimated travel time
    - One-sentence highlight

- **Output:** A concise, user-friendly list of recommended destinations.

In [70]:
# Summarizer Agent: Its job is to summarize the text it receives.
destination_agent = Agent(
    name="destination_agent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # The instruction is modified to request a bulleted list for a clear output format.
    instruction="""
        You are a travel assistant AI.
        Given the user's current location, find nearby tourist attractions (destinations).
        Summarize each destination with:
        - Name (in English)
        - Type (landmark, nature, cultural site, etc.)
        - Distance or estimated travel time
        - One-sentence highlight (why it's interesting)

        Return the summary in a concise, user-friendly format.""",
        tools=[google_search],
    output_key="destination_summary"
)

print("✅ destination_agent created.")

✅ destination_agent created.


## 3. budget agent


This agent specializes in estimating the costs associated with visiting selected tourist destinations.

- **Purpose:** Retrieve or estimate entrance fees and additional costs (e.g., transport, meals) for each destination.
- **How it works:** Uses available information to calculate the total estimated cost for the trip, presenting a clear summary.
- **Output:** For each destination, provides:
    - Destination name
    - Entrance fee (in local currency)
    - Optional additional costs
    - Total estimated cost

In [71]:
# Summarizer Agent: Its job is to summarize the text it receives.
budget_agent = Agent(
    name="budget_agent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # The instruction is modified to request a bulleted list for a clear output format.
    instruction="""
    You are a Budget Agent AI.

    Given a list of tourist destinations (with location and weather already considered),
    your tasks are:
    1. Retrieve or estimate the entrance fee for each destination.
    2. Include any additional costs if specified (e.g., transport, meals).
    3. Calculate the total estimated cost for the trip.
    4. Present the result in a clear, user-friendly summary.

    Output must include:
    - Destination name
    - Entrance fee (in local currency)
    - Optional additional costs
    - Total estimated cost

    Return the summary under the key 'budget_summary'.
    """,
    output_key="budget_summary"
)

print("✅ budget_agent created.")

✅ budget_agent created.


# Agent Coordinator

The main agent, **vacation_coordinator**, orchestrates the entire travel planning workflow by coordinating specialized sub-agents:

- **location_weather_agent**: Retrieves current weather and location details.
- **destination_agent**: Suggests and summarizes nearby tourist attractions, filtered by weather.
- **budget_agent**: Estimates entrance fees and total trip costs for selected destinations.

The main agent ensures a seamless, personalized vacation plan by integrating the outputs of all sub-agents and presenting a clear summary to the user.

In [72]:
vacation_coordinator_agent = Agent(
    name="vacation_coordinator",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # This instruction tells the root agent HOW to use its tools (which are the other agents).
    instruction="""
    You are a Vacation Coordinator AI agent.

    Workflow:
    1. Ask the user for their current location, or retrieve the user's location from their info.
    2. Send the location to the location_weather_agent to retrieve weather conditions.
    3. Pass both location and weather data to the destination_agent to find suitable tourist attractions.
    - Ensure destinations are filtered based on weather (e.g., avoid outdoor spots in heavy rain).
    4. Forward the selected destinations to the budget_agent.
    - Request entrance fees and calculate the total estimated cost for the trip.
    5. Return a final summary to the user including:
    - Location
    - Weather condition
    - Recommended destinations
    - Estimated entrance fees and total cost

    Output must be structured under the key 'vacation_plan'.
    """,
    output_key="vacation_plan",
    # We wrap the sub-agents in `AgentTool` to make them callable tools for the root agent.
    tools=[AgentTool(root_agent), AgentTool(location_weather_agent), AgentTool(destination_agent), AgentTool(budget_agent)],
)

print("✅ vacation_coordinator_agent created.")

✅ vacation_coordinator_agent created.


# Agent Test

In [73]:
runner = InMemoryRunner(agent=vacation_coordinator_agent)
response = await runner.run_debug(
    "hello, currently I'm in Yogyakarta and planning a vacation. Can you help me create a travel plan? my budget is 1 million rupiah"
)


 ### Created new session: debug_session_id

User > hello, currently I'm in Yogyakarta and planning a vacation. Can you help me create a travel plan? my budget is 1 million rupiah


vacation_coordinator > Hello! I've put together a vacation plan for you based on your location in Yogyakarta and your budget of 1 million rupiah.

**Location:** Yogyakarta
**Weather:** Currently cloudy and humid, with a chance of rain throughout the week. Temperatures will range between 71°F (22°C) and 87°F (31°C).

**Recommended Destinations (Indoor Attractions):**

*   **Taman Pintar Science Park:** An educational park with indoor attractions suitable for learning and fun.
*   **Ullen Sentalu Museum:** A cultural museum in Kaliurang showcasing Javanese history and culture.
*   **Museum Dirgantara Yogyakarta:** An air force museum featuring a collection of aircraft and aviation memorabilia.

**Estimated Costs:**

*   **Taman Pintar Science Park:** Approximately 50,000 IDR (including potential additional costs)
*   **Ullen Sentalu Museum:** Approximately 50,000 IDR (including optional guide services)
*   **Museum Dirgantara Yogyakarta:** Approximately 15,000 IDR

**Total Estimated Entr